In [1]:
import requests
import pandas as pd
import time
import logging
from datetime import datetime, timedelta

# Thiết lập logging
logging.basicConfig(level=logging.INFO)

# Danh sách 10 mã coin cần lấy dữ liệu
SYMBOLS = ["BNBUSDT", "BTCUSDT", "ETHUSDT", "XRPUSDT", "SOLUSDT",
           "LTCUSDT", "ETCUSDT", "PEPEUSDT", "DOGEUSDT", "ADAUSDT"]

def fetch_binance_data(symbol, start_time, end_time, interval="5m"):
    """
    Lấy dữ liệu lịch sử từ Binance API.
    
    :param symbol: Mã coin (ví dụ: BTCUSDT)
    :param start_time: Thời gian bắt đầu (timestamp in milliseconds)
    :param end_time: Thời gian kết thúc (timestamp in milliseconds)
    :param interval: Khoảng thời gian (ví dụ: 1m, 5m, 1h)
    :return: DataFrame chứa dữ liệu lịch sử
    """
    url = "https://api.binance.com/api/v3/klines"
    all_data = []
    
    while start_time < end_time:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": start_time,
            "endTime": min(start_time + 1000 * 5 * 60 * 1000, end_time),  # Lấy tối đa 1000 nến mỗi lần
            "limit": 1000
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if not data:
                break
            all_data.extend(data)
            start_time = data[-1][6] + 1  # Lấy timestamp của nến cuối cùng + 1ms
            logging.info(f"✅ Fetched {len(data)} rows for {symbol}. Continuing...")
            time.sleep(0.5)  # Tránh bị giới hạn API
        else:
            logging.error(f"❌ Failed to fetch data for {symbol}: {response.status_code}, {response.text}")
            break

    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(all_data, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
    df = df[["open_time", "open", "high", "low", "close", "volume", "close_time"]]
    return df

def format_data_for_postgres(df, symbol):
    """
    Định dạng dữ liệu giống như khi đưa vào PostgreSQL.
    
    :param df: DataFrame chứa dữ liệu lịch sử
    :param symbol: Mã coin
    :return: DataFrame đã được định dạng
    """
    df["symbol"] = symbol
    df.rename(columns={"close_time": "time"}, inplace=True)
    df["time"] = (df["time"] + pd.Timedelta(milliseconds=1)).dt.floor("S")
    df["time"] = df["time"].dt.strftime("%Y-%m-%d %H:%M:%S.%f").str[:-3]
    df = df[["symbol", "time", "open", "high", "low", "close", "volume"]]
    df["open"] = df["open"].astype(float)
    df["high"] = df["high"].astype(float)
    df["low"] = df["low"].astype(float)
    df["close"] = df["close"].astype(float)
    df["volume"] = df["volume"].astype(float)
    return df

if __name__ == "__main__":
    # Lấy thời gian hiện tại và 1 năm trước
    end_time = int(datetime.now().timestamp() * 1000)  # Thời gian hiện tại (timestamp in ms)
    start_time = int((datetime.now() - timedelta(days=365)).timestamp() * 1000)  # 1 năm trước (timestamp in ms)

    all_data = []

    for symbol in SYMBOLS:
        logging.info(f"🔍 Fetching historical data for {symbol} from {start_time} to {end_time}...")
        df = fetch_binance_data(symbol, start_time, end_time, interval="5m")
        if not df.empty:
            formatted_df = format_data_for_postgres(df, symbol)
            all_data.append(formatted_df)
            logging.info(f"✅ Fetched {len(formatted_df)} rows of data for {symbol}.")

    # Gộp dữ liệu từ tất cả các symbol
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        output_file = "crypto_history_5m_last_1_year_final.csv"
        final_df.to_csv(output_file, index=False)
        logging.info(f"✅ All data saved to {output_file}.")

INFO:root:🔍 Fetching historical data for BNBUSDT from 1715651337521 to 1747187337521...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...
INFO:root:✅ Fetched 1000 rows for BNBUSDT. Continuing...


KeyboardInterrupt: 